In [2]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import shutil
import random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

import time
import math
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [58]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda:0


In [59]:
# image preprocess
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485,0.456,0.406],
                                                           [0.229,0.224,0.225])
                                     ])

test_transform = transforms.Compose([transforms.Resize(224),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485,0.456,0.406],
                                         std=[0.229,0.224,0.225])
                                     ])


In [60]:
dataset_dir = 'SplitDataset'
train_path = os.path.join(dataset_dir, 'train')
test_path = os.path.join(dataset_dir, 'test')
train_dataset = datasets.ImageFolder(train_path, train_transform)
test_dataset = datasets.ImageFolder(test_path, test_transform)

In [61]:
# define dataloader
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset,
                          batch_size = BATCH_SIZE,
                          shuffle = True
                         )
test_loader = DataLoader(test_dataset,
                          batch_size = BATCH_SIZE,
                          shuffle = False
                         )

In [62]:
# check the images and labels in one batch
images, labels = next(iter(train_loader))

In [63]:
images.shape

torch.Size([64, 3, 224, 224])

In [64]:
class CNNmodel(nn.Module): 
    def __init__(self):
        super(CNNmodel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(512 * 14 * 14, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 2))

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.dropout(x)
        x = self.cnn(x)
        #print(x.shape) # torch.Size([64, 512, 14, 14])
        x = x.view(-1, 512 * 14 * 14)
        x = self.fc(x)
        return x
 
model = CNNmodel().to(device)

# The loss (or error) is measured as a number between 0 and 1, with 0 being a perfect model. 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-5, momentum=0.9, weight_decay=5e-4)

In [65]:
# train loop
epochs = 2000
time_list = []
train_loss_list = []
for epoch in range(epochs):
    train_loss = []
    start_time = time.time()
    for data in train_loader:
        images, labels = data[0].to(device), data[1].to(device)
 
        outputs = model(images)
        loss = criterion(outputs, labels)
        train_loss.append(float(loss.data.mean()))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss_list.append(np.mean(train_loss))
    total_time = time.time() - start_time
    time_list.append(total_time)
    
    print('Epoch %d loss: %.3f' % (epoch, np.mean(train_loss)))


Epoch 0 loss: 0.685
Epoch 1 loss: 0.682
Epoch 2 loss: 0.687
Epoch 3 loss: 0.686
Epoch 4 loss: 0.674
Epoch 5 loss: 0.678
Epoch 6 loss: 0.684
Epoch 7 loss: 0.680
Epoch 8 loss: 0.679
Epoch 9 loss: 0.678
Epoch 10 loss: 0.671
Epoch 11 loss: 0.678
Epoch 12 loss: 0.668
Epoch 13 loss: 0.675
Epoch 14 loss: 0.671
Epoch 15 loss: 0.672
Epoch 16 loss: 0.667
Epoch 17 loss: 0.668
Epoch 18 loss: 0.670
Epoch 19 loss: 0.665
Epoch 20 loss: 0.668
Epoch 21 loss: 0.665
Epoch 22 loss: 0.670
Epoch 23 loss: 0.670
Epoch 24 loss: 0.664
Epoch 25 loss: 0.667
Epoch 26 loss: 0.663
Epoch 27 loss: 0.662
Epoch 28 loss: 0.668
Epoch 29 loss: 0.670
Epoch 30 loss: 0.653
Epoch 31 loss: 0.663
Epoch 32 loss: 0.662
Epoch 33 loss: 0.656
Epoch 34 loss: 0.659
Epoch 35 loss: 0.657
Epoch 36 loss: 0.665
Epoch 37 loss: 0.660
Epoch 38 loss: 0.659
Epoch 39 loss: 0.659
Epoch 40 loss: 0.655
Epoch 41 loss: 0.657
Epoch 42 loss: 0.658
Epoch 43 loss: 0.660
Epoch 44 loss: 0.660
Epoch 45 loss: 0.656
Epoch 46 loss: 0.659
Epoch 47 loss: 0.650
Ep

Epoch 378 loss: 0.651
Epoch 379 loss: 0.636
Epoch 380 loss: 0.636
Epoch 381 loss: 0.622
Epoch 382 loss: 0.626
Epoch 383 loss: 0.647
Epoch 384 loss: 0.643
Epoch 385 loss: 0.641
Epoch 386 loss: 0.621
Epoch 387 loss: 0.645
Epoch 388 loss: 0.635
Epoch 389 loss: 0.617
Epoch 390 loss: 0.637
Epoch 391 loss: 0.635
Epoch 392 loss: 0.633
Epoch 393 loss: 0.641
Epoch 394 loss: 0.644
Epoch 395 loss: 0.623
Epoch 396 loss: 0.643
Epoch 397 loss: 0.629
Epoch 398 loss: 0.633
Epoch 399 loss: 0.626
Epoch 400 loss: 0.624
Epoch 401 loss: 0.622
Epoch 402 loss: 0.635
Epoch 403 loss: 0.606
Epoch 404 loss: 0.624
Epoch 405 loss: 0.629
Epoch 406 loss: 0.616
Epoch 407 loss: 0.627
Epoch 408 loss: 0.644
Epoch 409 loss: 0.632
Epoch 410 loss: 0.617
Epoch 411 loss: 0.631
Epoch 412 loss: 0.629
Epoch 413 loss: 0.633
Epoch 414 loss: 0.642
Epoch 415 loss: 0.641
Epoch 416 loss: 0.633
Epoch 417 loss: 0.626
Epoch 418 loss: 0.623
Epoch 419 loss: 0.647
Epoch 420 loss: 0.638
Epoch 421 loss: 0.657
Epoch 422 loss: 0.653
Epoch 423 

Epoch 751 loss: 0.604
Epoch 752 loss: 0.641
Epoch 753 loss: 0.622
Epoch 754 loss: 0.606
Epoch 755 loss: 0.618
Epoch 756 loss: 0.641
Epoch 757 loss: 0.620
Epoch 758 loss: 0.635
Epoch 759 loss: 0.603
Epoch 760 loss: 0.609
Epoch 761 loss: 0.614
Epoch 762 loss: 0.611
Epoch 763 loss: 0.615
Epoch 764 loss: 0.612
Epoch 765 loss: 0.606
Epoch 766 loss: 0.591
Epoch 767 loss: 0.627
Epoch 768 loss: 0.634
Epoch 769 loss: 0.617
Epoch 770 loss: 0.610
Epoch 771 loss: 0.595
Epoch 772 loss: 0.616
Epoch 773 loss: 0.603
Epoch 774 loss: 0.622
Epoch 775 loss: 0.610
Epoch 776 loss: 0.634
Epoch 777 loss: 0.627
Epoch 778 loss: 0.603
Epoch 779 loss: 0.616
Epoch 780 loss: 0.611
Epoch 781 loss: 0.623
Epoch 782 loss: 0.616
Epoch 783 loss: 0.612
Epoch 784 loss: 0.610
Epoch 785 loss: 0.632
Epoch 786 loss: 0.606
Epoch 787 loss: 0.615
Epoch 788 loss: 0.605
Epoch 789 loss: 0.622
Epoch 790 loss: 0.627
Epoch 791 loss: 0.611
Epoch 792 loss: 0.620
Epoch 793 loss: 0.623
Epoch 794 loss: 0.635
Epoch 795 loss: 0.627
Epoch 796 

Epoch 1119 loss: 0.605
Epoch 1120 loss: 0.599
Epoch 1121 loss: 0.581
Epoch 1122 loss: 0.607
Epoch 1123 loss: 0.599
Epoch 1124 loss: 0.610
Epoch 1125 loss: 0.616
Epoch 1126 loss: 0.613
Epoch 1127 loss: 0.614
Epoch 1128 loss: 0.609
Epoch 1129 loss: 0.594
Epoch 1130 loss: 0.603
Epoch 1131 loss: 0.608
Epoch 1132 loss: 0.590
Epoch 1133 loss: 0.622
Epoch 1134 loss: 0.610
Epoch 1135 loss: 0.583
Epoch 1136 loss: 0.600
Epoch 1137 loss: 0.598
Epoch 1138 loss: 0.609
Epoch 1139 loss: 0.589
Epoch 1140 loss: 0.614
Epoch 1141 loss: 0.615
Epoch 1142 loss: 0.588
Epoch 1143 loss: 0.634
Epoch 1144 loss: 0.609
Epoch 1145 loss: 0.600
Epoch 1146 loss: 0.588
Epoch 1147 loss: 0.609
Epoch 1148 loss: 0.606
Epoch 1149 loss: 0.604
Epoch 1150 loss: 0.593
Epoch 1151 loss: 0.627
Epoch 1152 loss: 0.636
Epoch 1153 loss: 0.636
Epoch 1154 loss: 0.618
Epoch 1155 loss: 0.609
Epoch 1156 loss: 0.620
Epoch 1157 loss: 0.590
Epoch 1158 loss: 0.609
Epoch 1159 loss: 0.619
Epoch 1160 loss: 0.623
Epoch 1161 loss: 0.578
Epoch 1162 

Epoch 1476 loss: 0.614
Epoch 1477 loss: 0.602
Epoch 1478 loss: 0.578
Epoch 1479 loss: 0.581
Epoch 1480 loss: 0.614
Epoch 1481 loss: 0.584
Epoch 1482 loss: 0.611
Epoch 1483 loss: 0.620
Epoch 1484 loss: 0.579
Epoch 1485 loss: 0.583
Epoch 1486 loss: 0.587
Epoch 1487 loss: 0.589
Epoch 1488 loss: 0.587
Epoch 1489 loss: 0.570
Epoch 1490 loss: 0.577
Epoch 1491 loss: 0.581
Epoch 1492 loss: 0.608
Epoch 1493 loss: 0.581
Epoch 1494 loss: 0.565
Epoch 1495 loss: 0.579
Epoch 1496 loss: 0.616
Epoch 1497 loss: 0.602
Epoch 1498 loss: 0.578
Epoch 1499 loss: 0.601
Epoch 1500 loss: 0.612
Epoch 1501 loss: 0.596
Epoch 1502 loss: 0.575
Epoch 1503 loss: 0.603
Epoch 1504 loss: 0.612
Epoch 1505 loss: 0.580
Epoch 1506 loss: 0.577
Epoch 1507 loss: 0.581
Epoch 1508 loss: 0.574
Epoch 1509 loss: 0.603
Epoch 1510 loss: 0.593
Epoch 1511 loss: 0.588
Epoch 1512 loss: 0.582
Epoch 1513 loss: 0.606
Epoch 1514 loss: 0.601
Epoch 1515 loss: 0.591
Epoch 1516 loss: 0.552
Epoch 1517 loss: 0.596
Epoch 1518 loss: 0.593
Epoch 1519 

Epoch 1833 loss: 0.572
Epoch 1834 loss: 0.580
Epoch 1835 loss: 0.572
Epoch 1836 loss: 0.573
Epoch 1837 loss: 0.583
Epoch 1838 loss: 0.584
Epoch 1839 loss: 0.605
Epoch 1840 loss: 0.556
Epoch 1841 loss: 0.594
Epoch 1842 loss: 0.613
Epoch 1843 loss: 0.568
Epoch 1844 loss: 0.590
Epoch 1845 loss: 0.583
Epoch 1846 loss: 0.558
Epoch 1847 loss: 0.557
Epoch 1848 loss: 0.567
Epoch 1849 loss: 0.585
Epoch 1850 loss: 0.597
Epoch 1851 loss: 0.556
Epoch 1852 loss: 0.562
Epoch 1853 loss: 0.576
Epoch 1854 loss: 0.589
Epoch 1855 loss: 0.563
Epoch 1856 loss: 0.596
Epoch 1857 loss: 0.573
Epoch 1858 loss: 0.560
Epoch 1859 loss: 0.549
Epoch 1860 loss: 0.572
Epoch 1861 loss: 0.589
Epoch 1862 loss: 0.587
Epoch 1863 loss: 0.592
Epoch 1864 loss: 0.577
Epoch 1865 loss: 0.559
Epoch 1866 loss: 0.557
Epoch 1867 loss: 0.567
Epoch 1868 loss: 0.567
Epoch 1869 loss: 0.559
Epoch 1870 loss: 0.563
Epoch 1871 loss: 0.579
Epoch 1872 loss: 0.604
Epoch 1873 loss: 0.579
Epoch 1874 loss: 0.570
Epoch 1875 loss: 0.554
Epoch 1876 

In [ ]:
# test
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
 
print('Accuracy on test set: %d%%' % (100*correct/total))

In [67]:
# accuracy on training set
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
 
print('Accuracy on training set: %d%%' % (100*correct/total))

Accuracy on training set: 72%


In [ ]:
x = np.linspace(0.0, epochs-1, epochs)

print('Total training time:', sum(time_list))
print('Average training time:', sum(time_list)/epochs)
plt.figure(figsize=(6,4))
plt.xlabel('Epoch')
plt.ylabel('Training time per epoch (s)')
plt.plot(x, time_list)
plt.grid()

Total training time: 37357.76709342003
Average training time: 18.678883546710015


In [ ]:
x = np.linspace(0.0, epochs-1, epochs)

plt.figure(figsize=(6,4))
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(x, train_loss_list)
plt.grid()